In [1]:
import pandas as pd
import ast
import numpy as np
from sklearn.model_selection import train_test_split

userDF = pd.read_csv("profiles.csv")
animeDF = pd.read_csv("animes.csv")
reviewsDF = pd.read_csv("reviews.csv")  
animeDF = animeDF.drop_duplicates(subset=['animeID'])
userDF = userDF.drop_duplicates(subset=['profile'])
reviewsDF = reviewsDF.drop_duplicates(subset=['profile','animeID'])
userDF = userDF[userDF['favorite_animes']!='[]']


#que ni por el putas se toque esta función, muchas gracias :)

def get_favorite_genres(animeIDS):
    animeIDS = ast.literal_eval(animeIDS)
    returnGenres = set()
    for animeID in animeIDS:
        anime_row = animeDF[animeDF['animeID'] == animeID]
        if not anime_row.empty:
            genres = anime_row['genres'].str.split(',')
            for genre in genres:
                returnGenres.update(genre)
    return list(returnGenres)



def getOpinion(score):
    # creamos el arreglo para categorizar si le gustó o no el anime
    opinion_array = np.zeros(3, dtype=int)
    
    if score < 5:
        opinion_array[0] = 1
    elif score >= 5 and score < 7:
        opinion_array[1] = 1
    elif score >= 7:
        opinion_array[2] = 1
    
    return opinion_array
    
def vectorizeGenres(genresList, allGenres,n):
    returnArray = np.zeros(n)
    for genre in genresList:
        for i in range(0,len(allGenres),1):
            if genre == allGenres[i]:
                returnArray[i] = 1
    return returnArray

In [2]:
#dataset de anime
animeDF

,animeID,title,genres
0,40960,Cheat Kusushi no Slow Life: Isekai ni Tsukurou...,"Comedy,Fantasy"
1,40957,Shin Chuuka Ichiban! 2,"Comedy,Shounen"
3,40956,Enen no Shouboutai: Ni no Shou,"Action,Supernatural,Shounen"
4,40938,Hige wo Soru. Soshite Joshikousei wo Hirou.,"Drama,Romance"
6,40936,Ore wo Suki nano wa Omae dake ka yo: Oretachi ...,"Comedy,Romance,School"
...,...,...,...
19221,8,Bouken Ou Beet,"Adventure,Fantasy,Shounen,Supernatural"
19222,7,Witch Hunter Robin,"Action,Magic,Police,Supernatural,Drama,Mystery"
19223,6,Trigun,"Action,Sci-Fi,Adventure,Comedy,Drama,Shounen"
19225,5,Cowboy Bebop: Tengoku no Tobira,"Action,Drama,Mystery,Sci-Fi,Space"


In [3]:
#dataset de usuarios
userDF

,profile,gender,birthday,favorite_animes,link
0,DesolatePsyche,Male,"Oct 2,1994","[33352,25013,5530,33674,1482,269,18245,2904,27...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10,2000","[11061,31964,853,20583,918,9253,34599,3588,136...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"[918,2904,11741,17074,23273,32281,9989,1943,512]",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"[5680,849,2904,3588,37349]",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30,1999","[4181,7791,9617,5680,2167,4382,849,235,35180,3...",https://myanimelist.net/profile/aManOfCulture99
...,...,...,...,...,...
77646,WeissYuki,NaN,NaN,"[36563,237,23273,32979,23283,12189,5114,4282]",https://myanimelist.net/profile/WeissYuki
77651,Akuteru,Male,"Aug 30,1993","[6624,20785,17265,28121]",https://myanimelist.net/profile/Akuteru
77652,Razielek,Male,"Nov 8,1986","[47,523,820,30,339]",https://myanimelist.net/profile/Razielek
77654,Mattierial,NaN,NaN,"[10087,21327,585,28851,32281,431,578,5114,1535]",https://myanimelist.net/profile/Mattierial


In [4]:
#dataset de reseñas
reviewsDF

,reviewID,profile,animeID,score
0,255938,DesolatePsyche,34096,8
1,259117,baekbeans,34599,10
2,253664,skrn,28891,7
3,8254,edgewalker00,2904,9
4,291149,aManOfCulture99,4181,10
...,...,...,...,...
182629,146535,iHitokage,2593,10
182631,13126,Onegai,2593,7
182633,127899,Murasa22,2593,10
182636,286852,srry4apologizng,2593,9


In [5]:
#el usuario no tiene géneros favoritos por default, lo que tiene es animes favoritos y aquellos animes son referencias al dataset de anime, el cual sí tiene generos, por ende debemos extraer estos
userDF['favorite_genres']=userDF['favorite_animes'].apply(get_favorite_genres)
userDF

,profile,gender,birthday,favorite_animes,link,favorite_genres
0,DesolatePsyche,Male,"Oct 2,1994","[33352,25013,5530,33674,1482,269,18245,2904,27...",https://myanimelist.net/profile/DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho..."
1,baekbeans,Female,"Nov 10,2000","[11061,31964,853,20583,918,9253,34599,3588,136...",https://myanimelist.net/profile/baekbeans,"[Parody, Shounen, Drama, Slice of Life, Samura..."
2,skrn,NaN,NaN,"[918,2904,11741,17074,23273,32281,9989,1943,512]",https://myanimelist.net/profile/skrn,"[Parody, Shounen, Drama, Slice of Life, Samura..."
3,edgewalker00,Male,Sep 5,"[5680,849,2904,3588,37349]",https://myanimelist.net/profile/edgewalker00,"[Parody, Fantasy, Adventure, School, Action, S..."
4,aManOfCulture99,Male,"Oct 30,1999","[4181,7791,9617,5680,2167,4382,849,235,35180,3...",https://myanimelist.net/profile/aManOfCulture99,"[Parody, Adventure, Police, School, Shounen, G..."
...,...,...,...,...,...,...
77646,WeissYuki,NaN,NaN,"[36563,237,23273,32979,23283,12189,5114,4282]",https://myanimelist.net/profile/WeissYuki,"[Shounen, Drama, Slice of Life, Adventure, Act..."
77651,Akuteru,Male,"Aug 30,1993","[6624,20785,17265,28121]",https://myanimelist.net/profile/Akuteru,"[Adventure, Fantasy, Action, School, Game, Sci..."
77652,Razielek,Male,"Nov 8,1986","[47,523,820,30,339]",https://myanimelist.net/profile/Razielek,"[Adventure, Dementia, Action, Sci-Fi, Drama, S..."
77654,Mattierial,NaN,NaN,"[10087,21327,585,28851,32281,431,578,5114,1535]",https://myanimelist.net/profile/Mattierial,"[Fantasy, Adventure, Police, Action, School, S..."


In [6]:
#de aqui solo nos importa el usuario y sus generos favoritos, el resto es relleno así que lo quitamos
columns_to_keep_from_user = ['profile','favorite_genres']
userDF = userDF[columns_to_keep_from_user]
userDF

,profile,favorite_genres
0,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho..."
1,baekbeans,"[Parody, Shounen, Drama, Slice of Life, Samura..."
2,skrn,"[Parody, Shounen, Drama, Slice of Life, Samura..."
3,edgewalker00,"[Parody, Fantasy, Adventure, School, Action, S..."
4,aManOfCulture99,"[Parody, Adventure, Police, School, Shounen, G..."
...,...,...
77646,WeissYuki,"[Shounen, Drama, Slice of Life, Adventure, Act..."
77651,Akuteru,"[Adventure, Fantasy, Action, School, Game, Sci..."
77652,Razielek,"[Adventure, Dementia, Action, Sci-Fi, Drama, S..."
77654,Mattierial,"[Fantasy, Adventure, Police, Action, School, S..."


In [7]:
#para que sea más facil de digerir para la máquina, convertimos la score en algo más simple: un arreglo de 3 posiciones, donde habrá solo 1 casilla con valor 1 y el resto tendrá 0, representando el primer indice que no le gustó, el segundo que le gustó mas o menos y el tercero que sí le gustó
reviewsDF['opinion'] = reviewsDF['score'].apply(getOpinion)
reviewsDF

,reviewID,profile,animeID,score,opinion
0,255938,DesolatePsyche,34096,8,"[0, 0, 1]"
1,259117,baekbeans,34599,10,"[0, 0, 1]"
2,253664,skrn,28891,7,"[0, 0, 1]"
3,8254,edgewalker00,2904,9,"[0, 0, 1]"
4,291149,aManOfCulture99,4181,10,"[0, 0, 1]"
...,...,...,...,...,...
182629,146535,iHitokage,2593,10,"[0, 0, 1]"
182631,13126,Onegai,2593,7,"[0, 0, 1]"
182633,127899,Murasa22,2593,10,"[0, 0, 1]"
182636,286852,srry4apologizng,2593,9,"[0, 0, 1]"


In [8]:
#relacionamos user con reseñas ya que ambos tienen la columna de profile, la cual es el nombre del usuario
user_and_review = pd.merge(reviewsDF, userDF, on='profile')
#quitamos el relleno y dejamos lo importante: usuario, generos, id de anime y opinion
user_and_review = user_and_review[['profile','favorite_genres','animeID','opinion']]
user_and_review

,profile,favorite_genres,animeID,opinion
0,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",34096,"[0, 0, 1]"
1,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",12403,"[0, 1, 0]"
2,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",35073,"[0, 1, 0]"
3,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",23225,"[0, 0, 1]"
4,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",2035,"[0, 0, 1]"
...,...,...,...,...
109300,WeissYuki,"[Shounen, Drama, Slice of Life, Adventure, Act...",2593,"[0, 0, 1]"
109301,Akuteru,"[Adventure, Fantasy, Action, School, Game, Sci...",2593,"[0, 0, 1]"
109302,Razielek,"[Adventure, Dementia, Action, Sci-Fi, Drama, S...",2593,"[0, 0, 1]"
109303,Mattierial,"[Fantasy, Adventure, Police, Action, School, S...",2593,"[0, 0, 1]"


In [9]:
#ahora unimos el dataset anterior con animeID, eliminamos también el relleno y nos quedan relacionados usuario y sus generos con anime y sus generos y la opinion del usuario para ese anime
all_merged = pd.merge(user_and_review,animeDF, on = 'animeID')
all_merged = all_merged[['profile','favorite_genres','title','genres', 'opinion']]
all_merged

,profile,favorite_genres,title,genres,opinion
0,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]"
1,Injenss,"[Parody, Shounen, Drama, Samurai, Historical, ...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]"
2,kokuborou,"[Parody, Shounen, Drama, Slice of Life, Samura...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]"
3,TheGostoso,"[Parody, Shounen, Drama, Slice of Life, Samura...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]"
4,PeterFromRussia,"[Parody, Shounen, Drama, Adventure, Action, Th...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]"
...,...,...,...,...,...
109285,Anime_is_Life3,"[Adventure, Fantasy, School, Drama, Comedy, Sh...",Yuki no Joou (TV),"Adventure,Drama,Fantasy,Shoujo","[0, 0, 1]"
109286,Funnysmily,"[Police, Adventure, Fantasy, Action, Shounen, ...",Higepiyo,Comedy,"[0, 0, 1]"
109287,Intumesce,"[Adventure, Action, Game, Shounen, Sci-Fi, Dra...",Tantei Opera Milky Holmes Movie: Gyakushuu no ...,"Comedy,Mystery,Super Power","[0, 0, 1]"
109288,paulthetallguy,"[Drama, Slice of Life, Action, Thriller, Comed...","Wake Up,Girls! Seishun no Kage","Drama,Music","[0, 0, 1]"


In [10]:
#ahora extraemos todos los generos de anime que existen
all_genres = animeDF['genres'].str.split(',').explode()
all_genres = set(all_genres)
for genre in all_genres:
    print(genre)

Parody
Shounen
Drama
Hentai
Slice of Life
Samurai
Historical
Horror
Adventure
Action
Game
Thriller
Shounen Ai
Comedy
Shoujo
Demons
Magic
Yuri
Dementia
School
Cars
Sci-Fi
Sports
Harem
Josei
Vampire
Music
Shoujo Ai
Martial Arts
Psychological
Seinen
Military
Fantasy
Police
Yaoi
Space
Mecha
Ecchi
Mystery
Kids
Romance
Super Power
Supernatural


In [11]:
#los pasamos a un arreglo para despues vectorizarlos (así le queda más fácil de digerir al modelo)
all_genres = np.array(list(all_genres))
n = len(all_genres)
all_merged['vectorized_favorite_genres'] = all_merged['favorite_genres'].apply(vectorizeGenres, args=(all_genres,n))

In [12]:
#convertimos los generos en un vector, cada posicion representa un genero y el 0 significa que ese genero no está presente, el 1 representa que sí está presente
all_merged['vectorized_anime_genres'] = all_merged['genres'].str.split(',').apply(vectorizeGenres, args=(all_genres, n))

In [13]:
all_merged

,profile,favorite_genres,title,genres,opinion,vectorized_favorite_genres,vectorized_anime_genres
0,DesolatePsyche,"[Shounen, Drama, Slice of Life, Historical, Ho...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
1,Injenss,"[Parody, Shounen, Drama, Samurai, Historical, ...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
2,kokuborou,"[Parody, Shounen, Drama, Slice of Life, Samura...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
3,TheGostoso,"[Parody, Shounen, Drama, Slice of Life, Samura...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
4,PeterFromRussia,"[Parody, Shounen, Drama, Adventure, Action, Th...",Gintama.,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...","[0, 0, 1]","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
109285,Anime_is_Life3,"[Adventure, Fantasy, School, Drama, Comedy, Sh...",Yuki no Joou (TV),"Adventure,Drama,Fantasy,Shoujo","[0, 0, 1]","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
109286,Funnysmily,"[Police, Adventure, Fantasy, Action, Shounen, ...",Higepiyo,Comedy,"[0, 0, 1]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
109287,Intumesce,"[Adventure, Action, Game, Shounen, Sci-Fi, Dra...",Tantei Opera Milky Holmes Movie: Gyakushuu no ...,"Comedy,Mystery,Super Power","[0, 0, 1]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
109288,paulthetallguy,"[Drama, Slice of Life, Action, Thriller, Comed...","Wake Up,Girls! Seishun no Kage","Drama,Music","[0, 0, 1]","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [14]:
#quitamos el relleno
columns_to_keep_from_all_merged = ['profile','vectorized_favorite_genres','title','vectorized_anime_genres','opinion']
all_merged = all_merged[columns_to_keep_from_all_merged]
all_merged

,profile,vectorized_favorite_genres,title,vectorized_anime_genres,opinion
0,DesolatePsyche,"[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",Gintama.,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
1,Injenss,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...",Gintama.,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
2,kokuborou,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",Gintama.,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
3,TheGostoso,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...",Gintama.,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
4,PeterFromRussia,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Gintama.,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
...,...,...,...,...,...
109285,Anime_is_Life3,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",Yuki no Joou (TV),"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0, 0, 1]"
109286,Funnysmily,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Higepiyo,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"
109287,Intumesce,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Tantei Opera Milky Holmes Movie: Gyakushuu no ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"
109288,paulthetallguy,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","Wake Up,Girls! Seishun no Kage","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"


In [15]:
#quitamos aun más relleno
trainDF = all_merged[['vectorized_favorite_genres','vectorized_anime_genres','opinion']]
trainDF

,vectorized_favorite_genres,vectorized_anime_genres,opinion
0,"[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
1,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
2,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
3,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
4,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0, 0, 1]"
...,...,...,...
109285,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0, 0, 1]"
109286,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"
109287,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"
109288,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1]"


In [16]:
trainDF = all_merged[['vectorized_favorite_genres','vectorized_anime_genres','opinion']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    trainDF[['vectorized_favorite_genres', 'vectorized_anime_genres']], 
    trainDF['opinion'], 
    test_size=0.20, 
    random_state=42
)

# Crear DataFrames para los conjuntos de test y train
test_df = pd.concat([X_test, y_test], axis=1)
train_df_csv = pd.concat([X_train, y_train], axis=1)

# Guardar en archivos CSV
test_df.to_csv('test.csv', index=False)
train_df_csv.to_csv('train.csv', index=False)

In [18]:
#Separar el dataset de Train
train_df = pd.read_csv('train.csv')
# Dividir los datos en conjuntos de entrenamiento (80%) y validación (20%)
train2_df, validation_df = train_test_split(
    train_df,
    test_size=0.20,
    random_state=42
)

# Guardar los datasets en archivos CSV
train2_df.to_csv('train2.csv', index=False)
validation_df.to_csv('validation.csv', index=False)